In [2]:
import pandas as pd

In [26]:
stock_data = pd.read_excel('D:\\rqadatas\\510300.xlsx', 
                           parse_dates=['时间'], 
                           date_parser=lambda date:date.split(',')[0])

In [27]:
stock_data.sort_values(by='时间', inplace=True)

In [28]:
stock_data.drop(columns=['总手','金额','换手%','成交次数'], inplace=True)

In [29]:
stock_data['ma_short'] = stock_data['收盘'].rolling(5, min_periods=1).mean()

In [30]:
stock_data['ma_long'] = stock_data['收盘'].rolling(60,min_periods=1).mean()

In [31]:
stock_data.loc[stock_data['ma_short'].shift(1) > stock_data['ma_long'].shift(1), 'position'] = 1

In [32]:
stock_data.loc[stock_data['ma_short'].shift(1) < stock_data['ma_long'].shift(1), 'position'] = 0

In [33]:
stock_data.fillna(0, inplace=True)

In [53]:
slippage=1.0/1000
commision_rate=2.5/1000
stock_data.loc[stock_data['position'] > stock_data['position'].shift(1), 'capital_rtn'] = \
    (stock_data['收盘'] / stock_data['开盘'] - 1) * \
    (1 - slippage - commision_rate) * (stock_data['position'] - stock_data['position'].shift(1)) + \
    stock_data['涨幅'] * stock_data['position'].shift(1)

In [60]:
stock_data['capital_rtn'].iloc[0]=0

C:\Users\yuanq\Anaconda3\envs\ptrhon36\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [64]:
stock_data.loc[stock_data['position'] < stock_data['position'].shift(1), 'capital_rtn'] = \
    (stock_data['开盘'] / stock_data['收盘'].shift(1) - 1) * \
    (1 - slippage - commision_rate) * \
    (stock_data['position'].shift(1) - stock_data['position']) + stock_data['涨幅'] * stock_data['position']

In [66]:
stock_data.loc[stock_data['position'] < stock_data['position'].shift(1), 'capital_rtn'] = stock_data['涨幅'] * stock_data['position']